In [1]:
import cv2
import os
import copy
import numpy as np

from ultis import *


In [3]:
#Đọc khung hình gốc của ảnh pano
compare_path = "D:/Xu ly anh/thucnghiem/v47"
compare_files = os.listdir(compare_path)
compare_files.sort()
compare_img = [cv2.imread(os.path.join(compare_path, img_file)) for img_file in compare_files]
#Đọc khung hình của video cần tracking
folder_path = "D:/Xu ly anh/thucnghiem/v44"
image_files = os.listdir(folder_path)
image_files.sort()
images = [cv2.imread(os.path.join(folder_path, img_file)) for img_file in image_files]
#Tạo kết quả output
output_folder = "D:/Xu ly anh/thucnghiem/v44_compare_akaze"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [4]:
#Đọc ma trận H
csv_file = "D:/Xu ly anh/thucnghiem/homographies_DJI_0044_sift.csv"
H = read_homographies_from_csv(csv_file)
#Tính toán kết quả đầu ra
output_width, output_height, min_x, min_y = calculate_output_size(images, H)
output_shape = (output_width, output_height)
result_origin = cv2.imread("D:/Xu ly anh/thucnghiem/result_DJI_0044_sift.jpg")
count = 0

In [5]:
"""
Duyệt qua từng ảnh và vẽ bbox lên ảnh pano
Dùng akaze để so sánh calculate_similarity_percentage_akaze
Dùng orb để so sánh: calculate_similarity_percentage_orb
Dùng sift để so sánh: calculate_similarity_percentage_sift
"""
for index,img_origin in enumerate(compare_img):
    img1 = cv2.cvtColor(img_origin, cv2.COLOR_BGR2GRAY)
    scores = []
    for i, img_file in enumerate(images):
        img2 = cv2.cvtColor(img_file, cv2.COLOR_BGR2GRAY)
        similarity_percentage = calculate_similarity_percentage_akaze(img1,img2)
        scores.append(similarity_percentage)

    sorted_indices = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)

    top_3_values = sorted_indices[:3]
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    top_3_values = [(item[0], item[1], colors[i]) for i, item in enumerate(top_3_values)]
    result = copy.deepcopy(result_origin)
    for index, position, color in top_3_values:
        img = images[index]
        img = draw_outer_bounding_box(img,color=color)
        warped_image = warp_image_to_zero(img, H, index, output_shape, min_x, min_y)
        result = np.maximum(result, warped_image)

    frame_path = os.path.join(output_folder, f"frame_{count:04d}_compare.jpg")
    result = concatenate_and_label_images(result,img_origin,"result_compare",f"frame_{count:04d}.jpg")
    cv2.imwrite(frame_path, result)
    count += 1
